In [0]:
%sql
select * from dev.demo_db.people_tbl

## Schema Evolution Approached

### Manual Approach

In [0]:
%sql
alter table dev.demo_db.people_tbl add columns(birthDate STRING)

In [0]:
%sql
INSERT INTO dev.demo_db.people_tbl
select id,fname firstName,lname lastName,dob birthDate
from json.`/Volumes/dev/demo_db/vol1/people.json`

In [0]:
%sql
select * from dev.demo_db.people_tbl

In [0]:
%sql
select * from json.`/Volumes/dev/demo_db/vol1/people_2.json`

### Manual Schenma Evolution - New Column in the Middle

In [0]:
%sql
alter table dev.demo_db.people_tbl add columns (phoneNumber string after lastName);

In [0]:
%sql
insert into dev.demo_db.people_tbl
select id,fname firstName,lname lastName,phone phoneNumber,dob birthDate
from json.`/Volumes/dev/demo_db/vol1/people_2.json`

In [0]:
%sql
select * from dev.demo_db.people_tbl

### Automatic Schema Evolution

In [0]:
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled","false")
spark.sql("select * from dev.demo_db.people_tbl").display()

### Automatic Schema Evolution - At Session Level

In [0]:
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

### Automatic Schema Evolution - New Column at the end

In [0]:
%sql
CREATE TABLE IF NOT EXISTS dev.demo_db.people_tbl_1

In [0]:
%sql
INSERT INTO dev.demo_db.people_tbl_1
select id,fname firstName,lname lastName,dob birthDate
from json.`/Volumes/dev/demo_db/vol1/people.json`

In [0]:
%sql
select * from dev.demo_db.people_tbl_1

In [0]:
%sql
insert into dev.demo_db.people_tbl_1
SELECT id,fname firstName,lname lastName,dob birthDate
from json.`/Volumes/dev/demo_db/vol1/people_2.json`

In [0]:
%sql
select * from dev.demo_db.people_tbl_1

In [0]:
%sql
select * from json.`/Volumes/dev/demo_db/vol1/people_2.json`

### Automatic Schema Evolution - New column in the middle

In [0]:
%sql
merge into dev.demo_db.people_tbl_1 tgt
using (select id,fname firstName,lname lastName,phone phoneNumber,dob birthDate from json.`/Volumes/dev/demo_db/vol1/people_2.json`)src
on tgt.id = src.id
WHEN NOT MATCHED then 
  INSERT *

In [0]:
%sql
select * from dev.demo_db.people_tbl_1

### Automatic Schema Evolution at Table Level

In [0]:
%sql
create table if not exists dev.demo_db.people_tbl_2

In [0]:
%sql
insert into dev.demo_db.people_tbl_2
select id,fname firstName,lname lastName
from json.`/Volumes/dev/demo_db/vol1/people.json`

In [0]:
%sql
select * from dev.demo_db.people_tbl_2

### Schema Evolution - New column at the end

In [0]:
# Read the schema from the existing Delta table
existing_schema = spark.table("dev.demo_db.people_tbl_2").schema

# Read the JSON data using the existing schema
people_2_df = (
    spark.read.format("json")
    .schema(existing_schema)
    .load("/Volumes/dev/demo_db/vol1/people_2.json")
)

# Write to the Delta table with mergeSchema enabled
(
    people_2_df.write
    .format("delta")
    .mode("append")
    .option("mergeSchema", "true")
    .saveAsTable("dev.demo_db.people_tbl_2")
)

### - Schema Evolution - New column in the middle

In [0]:
people_3_schema = "id INT,fname STRING,lname STRING,phone STRING,dob STRING"

people_3_df = (spark.read.format('json').schema(people_3_schema).load("/Volumes/dev/demo_db/vol1/people_3.json")
               .toDF("id","firstName","lastName","phoneNumber","birthDate"))

people_3_df.display()

In [0]:
%sql
select * from dev.demo_db.people_tbl_2

In [0]:
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled","true")

In [0]:
(people_3_df.write
 .format("delta")
 .mode("append")
 .option("mergeSchema", "true")
 .saveAsTable("dev.demo_db.people_tbl_2"))